In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
import seaborn as sns; sns.set(color_codes=True)
import numpy as np

In [2]:
species_id = pd.read_csv("~/code/skeeters/data/sample_genus_and_species.csv", header=0, index_col = 0)
species_id.rename(index={'CMS_001_RNA_A_S1':'CMS_001_RNA_A_S1_001'},inplace=True)
species_id.head()

,genus,species,corrected genus,corrected species
CMS_001_RNA_A_S1_001,Culex,erythrothorax,Culex,erythrothorax
CMS_002_10a_Rb_S119_L004,Culex,quinquefasciatus,Culex,quinquefasciatus
CMS_002_13a_Rb_S120_L004,Culex,quinquefasciatus,Culex,quinquefasciatus
CMS_002_16a_Rb_S121_L004,Aedes,aegypti,Aedes,aegypti
CMS_002_17a_Rb_S122_L004,Aedes,aegypti,Aedes,aegypti


In [3]:
dc_tsv = pd.read_csv("sketches/ska/10000000/distances.clusters.tsv", sep='\t')

In [5]:
dd_tsv = pd.read_csv("sketches/ska/10000000/distances.distances.tsv", sep='\t')
shape_dd_tsv = dd_tsv.shape

# Add prefix columns and size
dd_tsv["Sample 1 prefix"] = dd_tsv.apply(lambda row: row["Sample 1"].split("_10000000")[0], axis=1)
dd_tsv["Sample 2 prefix"] = dd_tsv.apply(lambda row: row["Sample 2"].split("_10000000")[0], axis=1)
dd_tsv["Size"] = dd_tsv.apply(lambda row: row["Sample 2"].split("_")[-1], axis=1)

dd_tsv.drop(['Sample 1', 'Sample 2'], axis=1)
dd_tsv.head()

# re-organize columns
dd_tsv = dd_tsv[["Sample 1 prefix", 
                 "Sample 2 prefix", 
                 "Size", "Matches", 
                 "Mismatches", 
                 "Jaccard Index",
                 "Mash-like distance",
                 "SNPs",
                 "SNP distance",
                ]]

# join on species data table
dd_tsv_w_labels = pd.merge(
    dd_tsv, 
    species_id[["genus", "species"]], 
    how='left',
    left_on="Sample 1 prefix", 
    right_index=True, 
)

# assert original data frame did not expand in row number
assert shape_dd_tsv[0] == dd_tsv_w_labels.shape[0]

In [6]:
dd_tsv_w_labels.head()

,Sample 1 prefix,Sample 2 prefix,Size,Matches,Mismatches,Jaccard Index,Mash-like distance,SNPs,SNP distance,genus,species
0,CMS_001_RNA_A_S1_001,CMS_002_10a_Rb_S119_L004_001,10000000,107591,796458,0.119010,0.049930,590,0.005484,Culex,erythrothorax
1,CMS_001_RNA_A_S1_001,CMS_002_13a_Rb_S120_L004_001,10000000,124812,898057,0.122021,0.049211,683,0.005472,Culex,erythrothorax
2,CMS_001_RNA_A_S1_001,CMS_002_16a_Rb_S121_L004_001,10000000,64767,1005138,0.060535,0.070005,591,0.009125,Culex,erythrothorax
3,CMS_001_RNA_A_S1_001,CMS_002_17a_Rb_S122_L004_001,10000000,67326,950575,0.066142,0.067318,585,0.008689,Culex,erythrothorax
4,CMS_001_RNA_A_S1_001,CMS_002_17b_Rb_S123_L004_001,10000000,68380,970402,0.065827,0.067462,593,0.008672,Culex,erythrothorax


In [11]:
dd_tsv.Matches/(dd_tsv.Matches + dd_tsv.Mismatches)

0        0.119010
1        0.122021
2        0.060535
3        0.066142
4        0.065827
5        0.063517
6        0.064399
7        0.068669
8        0.064455
9        0.063613
10       0.066260
11       0.120662
12       0.056889
13       0.084494
14       0.088217
15       0.072726
16       0.077090
17       0.051544
18       0.121451
19       0.119574
20       0.050486
21       0.063576
22       0.054121
23       0.060218
24       0.067787
25       0.053796
26       0.060898
27       0.062067
28       0.061644
29       0.057571
           ...   
12850    0.024023
12851    0.015832
12852    0.245919
12853    0.114472
12854    0.002864
12855    0.012772
12856    0.183811
12857    0.012768
12858    0.006988
12859    0.109297
12860    0.002921
12861    0.013868
12862    0.186785
12863    0.012053
12864    0.008433
12865    0.003796
12866    0.011383
12867    0.138618
12868    0.011318
12869    0.006974
12870    0.029951
12871    0.004310
12872    0.012425
12873    0.114993
12874    0

In [16]:
dd_tsv.Matches/(dd_tsv.Matches + dd_tsv.Mismatches).values

In [ ]:
distances_clusters_tsv.tail()

In [ ]:
# distances_distances_tsv.head(30)
data2d = distances_distances_tsv.pivot(columns="Sample 1", index="Sample 2", values="Mash-like distance")
print(data2d.shape)
data2d.head()

In [ ]:
data2d = data2d.loc[data2d.columns, :]
print(data2d.shape)
data2d.head()

In [ ]:
import numpy as np

np.fill_diagonal(data2d.values, 1)
data2d.head()

In [ ]:
data2d.T.head()

In [ ]:
data2d.fillna(0) + data2d.T.fillna(0) - np.diag(data2d.values.diagonal())

In [ ]:
data2d.values

In [ ]:
t = distances_distances_tsv[[""]]
#sns.pairplot()

In [ ]:
iris = sns.load_dataset("iris")
iris.head()

In [ ]:
from khtools import knn, sourmash_utils

In [ ]:
pd.__file__